In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import sys
from pathlib import Path

# this notebook is in notebooks — define project root
project_root = Path().resolve().parent
sys.path.append(str(project_root))

In [ ]:
import gradio as gr
import pandas as pd
from gdc_pipeline import execute_pipeline
from gdc_pipeline import setup_args
from gdc_pipeline import setup_models_and_data

In [ ]:
# init a dummy list
sys.argv = ['prog', '--question', 'dummy']
args = setup_args()

In [ ]:
args

In [ ]:
# setup models and data
qag_requirements = setup_models_and_data()

In [ ]:
def wrapped_execute_pipeline(question: str):
    df = pd.DataFrame({'questions' : [question]})
    print(f'Question received: {question}')
    try:
        result = execute_pipeline(
            df,
            qag_requirements.gdc_genes_mutations,
            qag_requirements.model,
            qag_requirements.tok,
            qag_requirements.intent_model,
            qag_requirements.intent_tok,
            qag_requirements.project_mappings,
            output_file_prefix=None
        )
    except Exception as e:
        result = 'Unable to execute GDC API, can you please retry with a template question?'
    return result

In [ ]:
question = 'What is the co-occurence frequency of somatic homozygous deletions in CDKN2A and CDKN2B in the mesothelioma project TCGA-MESO in the genomic data commons?'

In [ ]:
def visible_component(input_text):
    return gr.update(value="WHATEVER")


# Create Gradio interface
with gr.Blocks(title="GDC QAG MCP server") as demo:
    gr.Markdown(
        """
        # GDC QAG Service
        """
    )

    with gr.Row():
        query_input = gr.Textbox(
            lines = 3,
            label="Search Query",
            placeholder='e.g. "What is the co-occurence frequency of somatic homozygous deletions in CDKN2A and CDKN2B in the mesothelioma project TCGA-MESO in the genomic data commons?"',
            info="Required: Enter your search query",
        )

    search_button = gr.Button("Search", variant="primary")

    output = gr.Textbox(
        label="Query Result",
        lines=10,
        max_lines=25,
        info="The Result of the Query will appear here",
    )

    search_button.click(
        fn=wrapped_execute_pipeline,
        inputs=[query_input],
        outputs=output,
    )

In [ ]:
demo.launch(server_port=7862, show_api=True, mcp_server=True)